In [1]:
# Import packages
import requests as rq
import pandas as pd
import pulp as slvr

In [9]:
# Query Tables
url = 'https://copa-uniandes.github.io/2021-10-Proyecto-Model-Parameters/'
html = rq.get(url).content
df_list = pd.read_html(html)
CQ_ranking = df_list[0]
points_earned = df_list[1]
nationalities = df_list[2]

In [76]:
# Validate proper queries
# print(CQ_ranking)
# print(points_earned)
# print(nationalities)

   Unnamed: 0_level_0                Nombre               CQ 1  \
              Ranking    Unnamed: 1_level_1 Unnamed: 2_level_1   
0                   1         ROGLIC Primoz               2171   
1                   2         POGACAR Tadej               1173   
2                   3         VAN AERT Wout                510   
3                   4        FUGLSANG Jakob               1040   
4                   5  VAN DER POEL Mathieu                412   
..                ...                   ...                ...   
95                 96             HAIG Jack                754   
96                 97         BARDET Romain                474   
97                 98      BALLERINI Davide                120   
98                 99        CICCONE Giulio                654   
99                100     VERMEERSCH Gianni                193   

                 CQ 2               CQ 3               CQ 4  \
   Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
0              

In [10]:
# Create lp problem
prob = slvr.LpProblem("Bikes & More", slvr.LpMaximize)

# Create variables
# A rider i in Riders is chosen for period j in {1,2,3,4,5,6}
x = slvr.LpVariable.dicts("Chosen rider", [(i,j) for i in range(1,len(CQ_ranking[["Nombre"]]) + 1) for j in range(1,7)], 0, 1, "Binary" )
# Remaining budget for period j in {1,2,3,4,5,6}
r = slvr.LpVariable.dicts("Remaining budget", [j for j in range(1,7)], 0, 12000)
# Transfered rider i into the team in period j in {1,2,3,4,5,6}
t = slvr.LpVariable.dicts("Transfered rider", [(i,j) for i in range(1,len(CQ_ranking[["Nombre"]]) + 1) for j in range(1,7)], 0, 1, "Binary" )

D:\Software\Miniconda\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [11]:
# Create problem restrictions

# Exactly 10 riders must belong to the team
for j in range(1,7):
    prob += slvr.lpSum(x[i,j] for i in range(1,len(CQ_ranking[["Nombre"]]) + 1)) == 10

# Budget depends on available CQ and remaining budget
prob += slvr.lpSum(CQ_ranking[["CQ 1"]].iloc[i - 1].item() * x[i,1] for i in range(1,len(CQ_ranking[["Nombre"]]) + 1)) + r[1] == 12000

for j in range(2,7):
    prob += r[j-1] + slvr.lpSum(CQ_ranking[["CQ " + str(j)]].iloc[i - 1].item() * x[i,j - 1] for i in range(1,len(CQ_ranking[["Nombre"]]) + 1)) - slvr.lpSum(CQ_ranking[["CQ " + str(j)]].iloc[i - 1].item() * x[i,j] for i in range(1,len(CQ_ranking[["Nombre"]]) + 1)) == r[j]

# Can't transfer over 3 riders between one event and the next
for i in range(1,len(CQ_ranking[["Nombre"]]) + 1):
    for j in range(2,7):
        prob += t[i,j] >= x[i,j] - x[i,j - 1]

for j in range(2,7):
    prob += slvr.lpSum(t[i,j] for i in range(1,len(CQ_ranking[["Nombre"]]) + 1)) <= 3

In [12]:
# Función Objetivo
prob += slvr.lpSum(points_earned[["PE " + str(j)]].iloc[i - 1].item() * x[i,j] for i in range(1,len(points_earned[["Nombre"]]) + 1) for j in range(1,7))

In [13]:
# Solve the problem
prob.solve()

# Problem status
print(slvr.LpStatus[prob.status])

# Objective reached
print(slvr.value(prob.objective))

Optimal
16308.0


In [33]:
# Print the solution
#print("", end="\t")
for j in range(1,7):
    print(j, end = "\t")

#print(sum(x[i,j].varValue for i in range(1,len(CQ_ranking[["Nombre"]]) + 1) for j in range(1,7)))

with open('filename.txt', 'w') as f:
    print("")
    for i in range(1,len(CQ_ranking[["Nombre"]]) + 1):
        for j in range(1,7):
            if x[i,j].varValue >= 0.5:
                print(i * x[i,j].varValue, end = "\t")
            else:
                print("", end="\t")
        print("")


'''
for i in range(1,len(CQ_ranking[["Nombre"]]) + 1):
    for j in range(1,7):
        if x[i,j].varValue >= 1:
            print(i, end = "\t")
    print("")

'''

1	2	3	4	5	6	
						
						
						
				4.0	4.0	
						
6.0	6.0	6.0				
					7.0	
			8.0	8.0	8.0	
						
						
					11.0	
						
	13.0					
14.0	14.0	14.0				
						
						
						
						
						
						
21.0						
						
23.0						
						
						
						
						
						
						
			30.0	30.0		
		31.0	31.0	31.0		
						
						
						
						
						
						
38.0	38.0					
						
						
						
						
						
						
						
						
						
						
						
50.0	50.0	50.0	50.0	50.0		
			51.0	51.0	51.0	
						
						
						
55.0	55.0	55.0	55.0			
						
						
						
						
						
	61.0					
						
						
						
65.0	65.0	65.0	65.0	65.0	65.0	
	66.0	66.0	66.0	66.0	66.0	
				67.0	67.0	
						
						
		70.0				
						
						
						
						
					75.0	
76.0						
						
		78.0	78.0			
						
						
						
						
						
						
85.0	85.0	85.0	85.0			
						
						
						
						
						
						
						
						
						
						
						
						
				98.0	98.0	
						
						


'\nfor i in range(1,len(CQ_ranking[["Nombre"]]) + 1):\n    for j in range(1,7):\n        if x[i,j].varValue >= 1:\n            print(i, end = "\t")\n    print("")\n\n'